In [24]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [25]:
state_history_df = pd.read_csv("data\State_History.csv")
state_lookup_df = pd.read_csv("data\State_Lookup.csv")

In [26]:
state_history_df = state_history_df[state_history_df['month_date_yyyymm'] != 202509]

In [27]:
state_history_df['month_date_yyyymm'] = pd.to_datetime(state_history_df['month_date_yyyymm'].astype(str), format='%Y%m')

In [28]:
state_history_df

,month_date_yyyymm,state,state_id,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,median_days_on_market_mm,median_days_on_market_yy,new_listing_count,new_listing_count_mm,new_listing_count_yy,price_increased_count,price_increased_count_mm,price_increased_count_yy,price_increased_share,price_increased_share_mm,price_increased_share_yy,price_reduced_count,price_reduced_count_mm,price_reduced_count_yy,price_reduced_share,price_reduced_share_mm,price_reduced_share_yy,pending_listing_count,pending_listing_count_mm,pending_listing_count_yy,median_listing_price_per_square_foot,median_listing_price_per_square_foot_mm,median_listing_price_per_square_foot_yy,median_square_feet,median_square_feet_mm,median_square_feet_yy,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag
51,2025-08-01,Vermont,VT,509000.0,-0.0305,0.0182,2459,0.0321,0.1920,58,0.1837,0.1373,912,-0.1128,-0.0339,4,-0.7500,-0.6667,0.0009,-0.0025,-0.0023,732.0,0.1883,0.2282,0.1577,0.0265,-0.0122,1258.0,0.0328,1.1468,266.0,-0.0091,-0.0063,2007.0,-0.0048,0.0032,773721.0,-0.0104,-0.0102,3707,0.0299,0.3957,0.5116,0.0004,0.2275,0.0
52,2025-08-01,Iowa,IA,285000.0,-0.0170,-0.0494,11162,0.0318,0.2320,57,0.0654,0.0179,4048,-0.1020,0.0586,56,0.0000,-0.7308,0.0028,0.0000,-0.0089,3652.0,-0.0364,0.0616,0.1832,-0.0023,-0.0106,4831.0,-0.0543,-0.0002,170.0,-0.0074,-0.0183,1635.0,-0.0136,-0.0037,358384.0,-0.0154,0.0013,15859,0.0045,0.1532,0.4328,-0.0394,-0.1005,0.0
53,2025-08-01,Kansas,KS,299000.0,-0.0071,-0.0033,7803,0.0242,0.2329,51,0.0099,0.0200,3704,-0.0512,0.1197,156,0.3000,0.1471,0.0098,0.0022,-0.0001,2548.0,-0.0958,0.1314,0.1589,-0.0137,-0.0063,4555.0,-0.0447,0.1072,152.0,-0.0075,0.0035,2010.0,-0.0142,-0.0164,395554.0,-0.0218,-0.0071,12334,-0.0019,0.1771,0.5837,-0.0421,-0.0663,0.0
54,2025-08-01,Wisconsin,WI,399000.0,-0.0023,0.0233,12714,0.0325,0.0943,42,0.0909,0.1053,6568,-0.0520,0.0037,68,-0.1282,-0.2917,0.0027,-0.0003,-0.0014,4008.0,0.0086,0.1258,0.1603,0.0048,0.0074,6135.0,-0.0027,0.0645,216.0,-0.0064,0.0447,1811.0,-0.0025,-0.0077,508761.0,-0.0095,0.0288,18848,0.0251,0.0997,0.4825,-0.0170,-0.0135,0.0
55,2025-08-01,Hawaii,HI,750000.0,-0.0145,-0.0741,6214,-0.0297,0.1971,88,0.0602,0.1892,1392,-0.0531,-0.0492,36,-0.2500,0.2857,0.0039,-0.0011,0.0005,1500.0,0.1062,0.2058,0.1582,0.0152,0.0081,1709.0,0.0320,0.0530,696.0,0.0025,-0.0426,1100.0,-0.0036,-0.0274,1417907.0,-0.0010,-0.0876,7905,-0.0143,0.1615,0.2750,0.0164,-0.0376,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5656,2016-07-01,Minnesota,MN,244900.0,NaN,NaN,24687,NaN,NaN,56,NaN,NaN,11768,NaN,NaN,2128,NaN,NaN,0.0390,NaN,NaN,10516.0,NaN,NaN,0.1928,NaN,NaN,13525.0,NaN,NaN,124.0,NaN,NaN,2062.0,NaN,NaN,338598.0,NaN,NaN,38495,NaN,NaN,0.5479,NaN,NaN,NaN
5657,2016-07-01,Wisconsin,WI,188800.0,NaN,NaN,37250,NaN,NaN,64,NaN,NaN,9276,NaN,NaN,764,NaN,NaN,0.0149,NaN,NaN,8720.0,NaN,NaN,0.1707,NaN,NaN,4716.0,NaN,NaN,108.0,NaN,NaN,1720.0,NaN,NaN,260082.0,NaN,NaN,41623,NaN,NaN,0.1266,NaN,NaN,NaN
5658,2016-07-01,Illinois,IL,224900.0,NaN,NaN,61100,NaN,NaN,58,NaN,NaN,19576,NaN,NaN,2524,NaN,NaN,0.0228,NaN,NaN,28200.0,NaN,NaN,0.2552,NaN,NaN,29180.0,NaN,NaN,115.0,NaN,NaN,1850.0,NaN,NaN,349042.0,NaN,NaN,90015,NaN,NaN,0.4776,NaN,NaN,NaN
5659,2016-07-01,Indiana,IN,148500.0,NaN,NaN,28005,NaN,NaN,59,NaN,NaN,10640,NaN,NaN,304,NaN,NaN,0.0068,NaN,NaN,7672.0,NaN,NaN,0.1682,NaN,NaN,7001.0,NaN,NaN,80.0,NaN,NaN,1870.0,NaN,NaN,216906.0,NaN,NaN,34695,NaN,NaN,0.2500,NaN,NaN,NaN


In [29]:
def customDescription(df: pd.DataFrame, numeric_only: bool = False):
    if numeric_only:
        df = df.select_dtypes(include=np.number)
    
    desc = pd.DataFrame(index=df.columns.to_list())
    desc['type'] = df.dtypes
    desc['count'] = df.count()
    desc['nunique'] = df.nunique()
    desc['null'] = df.isnull().sum()
    
    # Calculate mode and handle multiple modes
    modes = df.mode()
    desc['mode'] = np.nan  # Default to NaN
    for col in df.columns:
        if len(modes[col].dropna()) == 1:  # Single mode exists
            desc.loc[col, 'mode'] = modes[col].iloc[0]
        else:  # Multiple modes
            desc.loc[col, 'mode'] = np.nan

    # Calculate least frequent value
    desc['least_frequent'] = np.nan  # Default to NaN
    for col in df.columns:
        value_counts = df[col].value_counts(dropna=False)
        if not value_counts.empty:
            least_freq_count = value_counts.min()  # Find the minimum frequency
            least_freq_values = value_counts[value_counts == least_freq_count].index
            
            if len(least_freq_values) == 1:  # If exactly one least frequent value exists
                desc.loc[col, 'least_frequent'] = least_freq_values[0]
            else:  # Multiple least frequent values
                desc.loc[col, 'least_frequent'] = np.nan
    
    # Handle numeric columns
    numeric_cols = df.select_dtypes(include=np.number)
    if not numeric_cols.empty:
        numeric_desc = numeric_cols.describe().T.drop(columns=['count', 'std', '25%', '50%', '75%'], axis=1)
        for col in numeric_cols.columns:
            desc.loc[col, 'mean'] = numeric_desc.loc[col, 'mean']
            desc.loc[col, 'min'] = numeric_desc.loc[col, 'min']
            desc.loc[col, 'max'] = numeric_desc.loc[col, 'max']
    
    # Handle datetime columns
    datetime_cols = df.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]'])
    for col in datetime_cols.columns:
        desc.loc[col, 'min'] = df[col].min()
        desc.loc[col, 'max'] = df[col].max()
    
    return desc

In [30]:
df = state_history_df.copy()

In [31]:
# customDescription(df)

In [32]:
pd.set_option('display.max_columns', None)

In [33]:
# Sort by state and month
df = df.sort_values(["state", "month_date_yyyymm"])

def recompute_mm(df, base_col, mm_col):
    # Compute pct_change
    df[mm_col] = df.groupby("state")[base_col].pct_change()

    # Replace inf values with the base_col (new month value)
    inf_mask = df[mm_col].replace([np.inf, -np.inf], np.nan).isna() & df[base_col].notna()
    df.loc[inf_mask, mm_col] = df.loc[inf_mask, base_col]

    return df

In [34]:
# You can loop over all *_mm cols systematically:
base_to_mm = {
    "median_listing_price": "median_listing_price_mm",
    "active_listing_count": "active_listing_count_mm",
    "median_days_on_market": "median_days_on_market_mm",
    "new_listing_count": "new_listing_count_mm",
    "price_increased_count": "price_increased_count_mm",
    "price_increased_share": "price_increased_share_mm",
    "price_reduced_count": "price_reduced_count_mm",
    "price_reduced_share": "price_reduced_share_mm",
    "pending_listing_count": "pending_listing_count_mm",
    "median_listing_price_per_square_foot": "median_listing_price_per_square_foot_mm",
    "median_square_feet": "median_square_feet_mm",
    "average_listing_price": "average_listing_price_mm",
    "total_listing_count": "total_listing_count_mm",
    "pending_ratio": "pending_ratio_mm"
}

for base, mm in base_to_mm.items():
    if base in df.columns and mm in df.columns:
        df = recompute_mm(df, base, mm)

In [35]:
df.reset_index(drop=True, inplace=True)
df.drop(columns=['quality_flag'], inplace=True, errors='ignore')
df

,month_date_yyyymm,state,state_id,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,median_days_on_market_mm,median_days_on_market_yy,new_listing_count,new_listing_count_mm,new_listing_count_yy,price_increased_count,price_increased_count_mm,price_increased_count_yy,price_increased_share,price_increased_share_mm,price_increased_share_yy,price_reduced_count,price_reduced_count_mm,price_reduced_count_yy,price_reduced_share,price_reduced_share_mm,price_reduced_share_yy,pending_listing_count,pending_listing_count_mm,pending_listing_count_yy,median_listing_price_per_square_foot,median_listing_price_per_square_foot_mm,median_listing_price_per_square_foot_yy,median_square_feet,median_square_feet_mm,median_square_feet_yy,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy
0,2016-07-01,Alabama,AL,185900.0,185900.000000,NaN,30468,30468.000000,NaN,78,78.000000,NaN,7756,7756.000000,NaN,368,368.000000,NaN,0.0094,0.009400,NaN,6132.0,6132.000000,NaN,0.1553,0.155300,NaN,1857.0,1857.000000,NaN,92.0,92.000000,NaN,1976.0,1976.000000,NaN,250718.0,250718.000000,NaN,32344,32344.000000,NaN,0.0609,0.060900,NaN
1,2016-08-01,Alabama,AL,185000.0,-0.004841,NaN,29926,-0.017789,NaN,83,0.064103,NaN,6884,-0.112429,NaN,452,0.228261,NaN,0.0117,0.244681,NaN,5864.0,-0.043705,NaN,0.1526,-0.017386,NaN,1905.0,0.025848,NaN,92.0,0.000000,NaN,1974.0,-0.001012,NaN,250649.0,-0.000275,NaN,31818,-0.016263,NaN,0.0636,0.044335,NaN
2,2016-09-01,Alabama,AL,184900.0,-0.000541,NaN,29557,-0.012330,NaN,85,0.024096,NaN,6676,-0.030215,NaN,424,-0.061947,NaN,0.0112,-0.042735,NaN,5790.0,-0.012619,NaN,0.1529,0.001966,NaN,1736.0,-0.088714,NaN,92.0,0.000000,NaN,1959.0,-0.007599,NaN,248452.0,-0.008765,NaN,31277,-0.017003,NaN,0.0587,-0.077044,NaN
3,2016-10-01,Alabama,AL,182795.0,-0.011385,NaN,28923,-0.021450,NaN,88,0.035294,NaN,6096,-0.086878,NaN,520,0.226415,NaN,0.0142,0.267857,NaN,5268.0,-0.090155,NaN,0.1447,-0.053630,NaN,1617.0,-0.068548,NaN,91.0,-0.010870,NaN,1950.0,-0.004594,NaN,250271.0,0.007321,NaN,30527,-0.023979,NaN,0.0559,-0.047700,NaN
4,2016-11-01,Alabama,AL,180500.0,-0.012555,NaN,27925,-0.034505,NaN,93,0.056818,NaN,5332,-0.125328,NaN,384,-0.261538,NaN,0.0102,-0.281690,NaN,3946.0,-0.250949,NaN,0.1061,-0.266759,NaN,1452.0,-0.102041,NaN,91.0,0.000000,NaN,1949.0,-0.000513,NaN,247238.0,-0.012119,NaN,29367,-0.037999,NaN,0.0520,-0.069767,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5605,2025-04-01,Wyoming,WY,476000.0,0.035008,0.0462,1945,0.065170,0.1421,57,-0.161765,-0.1172,942,0.201531,0.1135,16,0.333333,-0.2000,0.0046,0.121951,-0.0019,388.0,0.168675,0.2357,0.1135,0.072779,0.0121,597.0,0.189243,0.0093,209.0,0.004808,0.0716,2204.0,0.012868,-0.0267,1108065.0,-0.050301,0.2097,2535,0.093615,0.1128,0.3067,0.115678,-0.0404
5606,2025-05-01,Wyoming,WY,495000.0,0.039916,0.0623,2208,0.135219,0.1820,51,-0.105263,0.0000,1028,0.091295,0.1762,12,-0.250000,0.0000,0.0033,-0.282609,-0.0004,520.0,0.340206,0.3000,0.1354,0.192952,0.0166,615.0,0.030151,-0.0144,211.0,0.009569,0.0443,2216.0,0.005445,-0.0353,1064091.0,-0.039685,0.1772,2816,0.110848,0.1373,0.2785,-0.091947,-0.0555
5607,2025-06-01,Wyoming,WY,496741.0,0.003517,0.0458,2519,0.140851,0.1644,50,-0.019608,0.0638,1146,0.114786,0.1506,14,0.166667,0.7500,0.0032,-0.030303,0.0011,682.0,0.311538,0.2724,0.1597,0.179468,0.0161,649.0,0.055285,0.0973,217.0,0.028436,0.0210,2209.0,-0.003159,-0.0222,1166952.0,0.096666,0.1760,3156,0.120739,0.1529,0.2575,-0.075404,-0.0157
5608,2025-07-01,Wyoming,WY,496647.0,-0.000189,0.0137,2771,0.100040,0.1916,52,0.040000,0.0097,1004,-0.123909,0.0681,14,0.000000,0.1667,0.0033,0.031250,0.0001,792.0,0.161290,0.3113,0.1817,0.137758,0.0266,625.0,-0.03698

In [36]:
# customDescription(df)

In [37]:
df2 = df[df['month_date_yyyymm'] >= '2017-07-01']

In [38]:
df2[['price_increased_count_yy','pending_listing_count','pending_listing_count_yy','pending_ratio','pending_ratio_yy']] = \
    df2[['price_increased_count_yy','pending_listing_count','pending_listing_count_yy','pending_ratio','pending_ratio_yy']].fillna(0)

In [39]:
# # Mapping dictionary: state -> (Region, Division)
# state_to_region_division = {
#     # Northeast - New England
#     "Connecticut": ("Northeast", "New England"),
#     "Maine": ("Northeast", "New England"),
#     "Massachusetts": ("Northeast", "New England"),
#     "New Hampshire": ("Northeast", "New England"),
#     "Rhode Island": ("Northeast", "New England"),
#     "Vermont": ("Northeast", "New England"),

#     # Northeast - Middle Atlantic
#     "New Jersey": ("Northeast", "Middle Atlantic"),
#     "New York": ("Northeast", "Middle Atlantic"),
#     "Pennsylvania": ("Northeast", "Middle Atlantic"),

#     # Midwest - East North Central
#     "Illinois": ("Midwest", "East North Central"),
#     "Indiana": ("Midwest", "East North Central"),
#     "Michigan": ("Midwest", "East North Central"),
#     "Ohio": ("Midwest", "East North Central"),
#     "Wisconsin": ("Midwest", "East North Central"),

#     # Midwest - West North Central
#     "Iowa": ("Midwest", "West North Central"),
#     "Kansas": ("Midwest", "West North Central"),
#     "Minnesota": ("Midwest", "West North Central"),
#     "Missouri": ("Midwest", "West North Central"),
#     "Nebraska": ("Midwest", "West North Central"),
#     "North Dakota": ("Midwest", "West North Central"),
#     "South Dakota": ("Midwest", "West North Central"),

#     # South - South Atlantic
#     "Delaware": ("South", "South Atlantic"),
#     "Florida": ("South", "South Atlantic"),
#     "Georgia": ("South", "South Atlantic"),
#     "Maryland": ("South", "South Atlantic"),
#     "North Carolina": ("South", "South Atlantic"),
#     "South Carolina": ("South", "South Atlantic"),
#     "Virginia": ("South", "South Atlantic"),
#     "West Virginia": ("South", "South Atlantic"),
#     "District of Columbia": ("South", "South Atlantic"),

#     # South - East South Central
#     "Alabama": ("South", "East South Central"),
#     "Kentucky": ("South", "East South Central"),
#     "Mississippi": ("South", "East South Central"),
#     "Tennessee": ("South", "East South Central"),

#     # South - West South Central
#     "Arkansas": ("South", "West South Central"),
#     "Louisiana": ("South", "West South Central"),
#     "Oklahoma": ("South", "West South Central"),
#     "Texas": ("South", "West South Central"),

#     # West - Mountain
#     "Arizona": ("West", "Mountain"),
#     "Colorado": ("West", "Mountain"),
#     "Idaho": ("West", "Mountain"),
#     "Montana": ("West", "Mountain"),
#     "Nevada": ("West", "Mountain"),
#     "New Mexico": ("West", "Mountain"),
#     "Utah": ("West", "Mountain"),
#     "Wyoming": ("West", "Mountain"),

#     # West - Pacific
#     "Alaska": ("West", "Pacific"),
#     "California": ("West", "Pacific"),
#     "Hawaii": ("West", "Pacific"),
#     "Oregon": ("West", "Pacific"),
#     "Washington": ("West", "Pacific"),
# }

# # Create Region and Division columns
# df2["Region"] = df2["state"].map(lambda x: state_to_region_division.get(x, (None, None))[0])
# df2["Division"] = df2["state"].map(lambda x: state_to_region_division.get(x, (None, None))[1])

In [40]:
df2['month_date_yyyymm'] = pd.to_datetime(df2['month_date_yyyymm'])
df2['year'] = df2['month_date_yyyymm'].dt.year
df2['month'] = df2['month_date_yyyymm'].dt.month

df2.drop(columns=['month_date_yyyymm'], inplace=True, errors='ignore')

In [41]:
df2 = df2.sort_values(["year", "month", "state"])

In [42]:
df2.drop(columns=['state'], inplace=True, errors='ignore')

In [43]:
df2 = df2[['year', 'month', 'state_id'] + [col for col in df2.columns if col not in ['year', 'month', 'state_id']]]
df2

,year,month,state_id,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,median_days_on_market_mm,median_days_on_market_yy,new_listing_count,new_listing_count_mm,new_listing_count_yy,price_increased_count,price_increased_count_mm,price_increased_count_yy,price_increased_share,price_increased_share_mm,price_increased_share_yy,price_reduced_count,price_reduced_count_mm,price_reduced_count_yy,price_reduced_share,price_reduced_share_mm,price_reduced_share_yy,pending_listing_count,pending_listing_count_mm,pending_listing_count_yy,median_listing_price_per_square_foot,median_listing_price_per_square_foot_mm,median_listing_price_per_square_foot_yy,median_square_feet,median_square_feet_mm,median_square_feet_yy,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy
12,2017,7,AL,198060.0,-0.004223,0.0654,26519,0.002343,-0.1296,75,0.027397,-0.0385,7772,0.019680,0.0021,552,-0.073826,0.5000,0.0155,-0.043210,0.0061,6064.0,0.024670,-0.0111,0.1704,0.055108,0.0151,2092.0,-0.066071,0.1265,95.0,0.000000,0.0378,2022.0,0.000495,0.0233,264342.0,-0.000707,0.0543,28563,-0.004288,-0.1169,0.0789,-0.067376,0.0179
122,2017,7,AK,279375.0,-0.006667,-0.0333,3710,0.092784,-0.0196,53,0.104167,-0.1452,1368,-0.066849,0.1176,40,0.666667,0.1111,0.0077,0.571429,0.0002,1232.0,0.120000,-0.0096,0.2407,0.084723,-0.0009,0.0,0.000000,0.0000,164.0,0.000000,0.0015,1750.0,0.005747,-0.0278,327328.0,-0.003374,-0.0121,3710,0.092784,-0.0196,0.0000,0.000000,0.0000
232,2017,7,AZ,312000.0,-0.018868,0.0576,26807,-0.026015,-0.1148,56,0.037037,-0.0968,12744,-0.045822,0.0449,1064,0.015267,-0.1104,0.0224,0.051643,-0.0019,11256.0,0.012230,-0.0532,0.2329,0.029165,0.0029,8010.0,-0.073560,-0.0180,152.0,0.000000,0.0479,2107.0,-0.010333,0.0086,477781.0,-0.029825,0.0699,34886,-0.034965,-0.0948,0.2988,-0.048711,0.0294
342,2017,7,AR,169900.0,0.000000,0.0534,15717,-0.013309,-0.1057,73,-0.013514,-0.0519,4944,0.078534,0.1298,136,0.360000,0.3077,0.0057,0.295455,0.0016,2932.0,-0.014785,-0.0803,0.1275,0.000000,-0.0003,2784.0,-0.010309,-0.0259,92.0,0.000000,0.0384,1920.0,0.003135,0.0245,235007.0,0.003810,0.0652,18446,-0.013161,-0.0939,0.1771,0.002831,0.0145
452,2017,7,CA,549000.0,-0.001637,0.0660,71686,0.037319,-0.1431,40,0.081081,-0.1111,44520,-0.058814,-0.0548,9168,0.088836,0.2133,0.0595,0.079855,0.0156,28188.0,0.168754,-0.0891,0.1853,0.177255,0.0047,35444.0,-0.034855,-0.0850,294.0,-0.006757,0.0478,1899.0,-0.005759,-0.0005,1030673.0,-0.016759,0.1016,106530,0.009907,-0.1262,0.4944,-0.069627,0.0314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5169,2025,8,VA,450000.0,-0.025917,0.0227,20949,0.011589,0.3085,44,0.023256,0.0732,10452,-0.027902,0.0965,392,0.082873,-0.2403,0.0095,0.055556,-0.0051,7964.0,-0.011665,0.3855,0.1925,0.131687,0.0271,10321.0,-0.044529,0.0231,230.0,-0.004329,0.0349,1982.0,-0.008008,-0.0090,632180.0,-0.017907,-0.0175,31114,-0.014850,0.2130,0.4927,-0.055406,-0.1374
5279,2025,8,WA,649950.0,-0.014443,-0.0001,24525,0.012426,0.2941,51,0.133333,0.1333,9256,-0.115105,0.0557,460,0.040724,-0.1418,0.0111,0.144330,-0.0030,9464.0,-0.062599,0.2342,0.2180,-0.034544,0.0191,9810.0,-0.000102,-0.0467,343.0,-0.017192,0.0086,1975.0,-0.003029,-0.0075,898753.0,-0.014194,-0.0010,34309,0.010158,0.1698,0.4000,-0.012346,-0.1430
5389,2025,8,WV,269000.0,-0.003704,0.1208,4087,0.073267,0.2017,53,0.000000,-0.0185,1940,0.021053,0.1687,44,0.047619,-0.4211,0.0053,0.081633,-0.0047,1376.0,0.061728,0.2374,0.1618,0.054072,0.0158,2423.0,-0.010212,-0.0370,145.0,-0.013605,0.0534,1774.0,-0.010045,0.0080,335827.0,-0.012918,0.0309,6516,0.042894,0.1022,0.5929,-0.077773,-0.1469
5499,2025,8,WI,399000.0,-0.002251,0.0233,12714,0.032483,0.0943,42,0.076923,0.1053,

In [44]:
df2 = pd.merge(df2, state_lookup_df[['state_id', 'state_num']], on='state_id', how='left')
df2.drop(columns=['state_id'], inplace=True, errors='ignore')
df2 = df2[['year', 'month', 'state_num'] + [col for col in df2.columns if col not in ['year', 'month', 'state_num']]]
df2

,year,month,state_num,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,median_days_on_market_mm,median_days_on_market_yy,new_listing_count,new_listing_count_mm,new_listing_count_yy,price_increased_count,price_increased_count_mm,price_increased_count_yy,price_increased_share,price_increased_share_mm,price_increased_share_yy,price_reduced_count,price_reduced_count_mm,price_reduced_count_yy,price_reduced_share,price_reduced_share_mm,price_reduced_share_yy,pending_listing_count,pending_listing_count_mm,pending_listing_count_yy,median_listing_price_per_square_foot,median_listing_price_per_square_foot_mm,median_listing_price_per_square_foot_yy,median_square_feet,median_square_feet_mm,median_square_feet_yy,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy
0,2017,7,1,198060.0,-0.004223,0.0654,26519,0.002343,-0.1296,75,0.027397,-0.0385,7772,0.019680,0.0021,552,-0.073826,0.5000,0.0155,-0.043210,0.0061,6064.0,0.024670,-0.0111,0.1704,0.055108,0.0151,2092.0,-0.066071,0.1265,95.0,0.000000,0.0378,2022.0,0.000495,0.0233,264342.0,-0.000707,0.0543,28563,-0.004288,-0.1169,0.0789,-0.067376,0.0179
1,2017,7,2,279375.0,-0.006667,-0.0333,3710,0.092784,-0.0196,53,0.104167,-0.1452,1368,-0.066849,0.1176,40,0.666667,0.1111,0.0077,0.571429,0.0002,1232.0,0.120000,-0.0096,0.2407,0.084723,-0.0009,0.0,0.000000,0.0000,164.0,0.000000,0.0015,1750.0,0.005747,-0.0278,327328.0,-0.003374,-0.0121,3710,0.092784,-0.0196,0.0000,0.000000,0.0000
2,2017,7,3,312000.0,-0.018868,0.0576,26807,-0.026015,-0.1148,56,0.037037,-0.0968,12744,-0.045822,0.0449,1064,0.015267,-0.1104,0.0224,0.051643,-0.0019,11256.0,0.012230,-0.0532,0.2329,0.029165,0.0029,8010.0,-0.073560,-0.0180,152.0,0.000000,0.0479,2107.0,-0.010333,0.0086,477781.0,-0.029825,0.0699,34886,-0.034965,-0.0948,0.2988,-0.048711,0.0294
3,2017,7,4,169900.0,0.000000,0.0534,15717,-0.013309,-0.1057,73,-0.013514,-0.0519,4944,0.078534,0.1298,136,0.360000,0.3077,0.0057,0.295455,0.0016,2932.0,-0.014785,-0.0803,0.1275,0.000000,-0.0003,2784.0,-0.010309,-0.0259,92.0,0.000000,0.0384,1920.0,0.003135,0.0245,235007.0,0.003810,0.0652,18446,-0.013161,-0.0939,0.1771,0.002831,0.0145
4,2017,7,5,549000.0,-0.001637,0.0660,71686,0.037319,-0.1431,40,0.081081,-0.1111,44520,-0.058814,-0.0548,9168,0.088836,0.2133,0.0595,0.079855,0.0156,28188.0,0.168754,-0.0891,0.1853,0.177255,0.0047,35444.0,-0.034855,-0.0850,294.0,-0.006757,0.0478,1899.0,-0.005759,-0.0005,1030673.0,-0.016759,0.1016,106530,0.009907,-0.1262,0.4944,-0.069627,0.0314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,2025,8,47,450000.0,-0.025917,0.0227,20949,0.011589,0.3085,44,0.023256,0.0732,10452,-0.027902,0.0965,392,0.082873,-0.2403,0.0095,0.055556,-0.0051,7964.0,-0.011665,0.3855,0.1925,0.131687,0.0271,10321.0,-0.044529,0.0231,230.0,-0.004329,0.0349,1982.0,-0.008008,-0.0090,632180.0,-0.017907,-0.0175,31114,-0.014850,0.2130,0.4927,-0.055406,-0.1374
4994,2025,8,48,649950.0,-0.014443,-0.0001,24525,0.012426,0.2941,51,0.133333,0.1333,9256,-0.115105,0.0557,460,0.040724,-0.1418,0.0111,0.144330,-0.0030,9464.0,-0.062599,0.2342,0.2180,-0.034544,0.0191,9810.0,-0.000102,-0.0467,343.0,-0.017192,0.0086,1975.0,-0.003029,-0.0075,898753.0,-0.014194,-0.0010,34309,0.010158,0.1698,0.4000,-0.012346,-0.1430
4995,2025,8,49,269000.0,-0.003704,0.1208,4087,0.073267,0.2017,53,0.000000,-0.0185,1940,0.021053,0.1687,44,0.047619,-0.4211,0.0053,0.081633,-0.0047,1376.0,0.061728,0.2374,0.1618,0.054072,0.0158,2423.0,-0.010212,-0.0370,145.0,-0.013605,0.0534,1774.0,-0.010045,0.0080,335827.0,-0.012918,0.0309,6516,0.042894,0.1022,0.5929,-0.077773,-0.1469
4996,2025,8,50,399000.0,-0.002251,0.0233,12714,0.032483,0.0943,42,0.076923,0.1053,6568,-0.05196

In [45]:
# Cast selected columns to int (fill NaN with 0 before conversion)
int_columns = [
    'state_num',
    'median_listing_price',
    'active_listing_count',
    'median_days_on_market',
    'new_listing_count',
    'price_increased_count',
    'price_reduced_count',
    'pending_listing_count',
    'median_listing_price_per_square_foot',
    'median_square_feet',
    'average_listing_price',
    'total_listing_count'
]
for col in int_columns:
    if col in df.columns:
        df2[col] = df2[col].astype('int32')

In [ ]:
df2.to_csv("data/State_Processed.csv", index=False)